Chapter 2. 시퀀스

문자열, 리스트, 바이트 시퀀스, 배열, XML 요소, 데이터베이스 결과 등에 모두 반복, 슬라이싱, 정렬, 연결 등의 공통된 연산을 적용할 수 있다.

유니코드 문자열 -> 4장
시퀀스형을 구현하는 방법 -> 10장

2.1 내장 시퀀스 개요

파이썬 표준 라이브러리는 C로 구현된 다음과 같은 시퀀스형을 제공

[컨테이너 시퀀스]
서로 다른 자료형의 항목들을 담을 수 있는 list, tuple, collections.deque 형
- 객체에 대한 참조를 담고 있음
[균일 시퀀스]
단 하나의 자료형만 담을 수 있는 str, bytes, bytearray, memoryview, array.array 형
- 자신의 메모리 공간에 각 항목의 값을 직접 담고 있음
- 컨테이너 시퀀스 보다 메모리를 더 적게 사용

다음과 같이 가변성에 따라 분류할 수도 있다.

[가변 시퀀스]
list, bytearray, array.array, collections.deque, memoryview 형

[불변 시퀀스]
tuple, str, bytes 형

2.2 지능형 리스트와 제너레이터 표현식

지능형 리스트(`listcomp`, list 형의 경우)나 제너레이터 표현식(`genexp`, 그 외의 시퀀스의 경우)을 사용하면 시퀀스를 가독성이 좋고 떄로는 실행 속도도 빠르게 생성가능

2.2.1 지능형 리스트와 가독성

In [1]:
# 예제 2-1) 문자열에서 유니코드 코드포인트 리스트 만들기 (ver.1)
symbols = '$#(@&$'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))

codes

[36, 35, 40, 64, 38, 36]

In [2]:
# 예제 2-2) 문자열에서 유니코드 코드포인트 리스트 만들기 (ver.2)
symbols = '$#(@&$'
codes = [ord(symbol) for symbol in symbols]

codes

[36, 35, 40, 64, 38, 36]

가독성: 예제 2-1) < 예제 2-2) - 의도를 명확히 보여줌

지능형 리스트는 오로지 새로운 리스트를 만드는 일만 한다.

주의)
- 생성된 리스트를 사용하지 않을 거라면 지능형 리스트 구문을 사용x
- 코드를 짧게 만들어야 한다. (지능형 리스트 구문이 두 줄 이상을 넘어가는 경우에는 코드를 분할하거나 for 문을 이용)

Tip) 파이썬에서는 `[],{},()` 안에서의 개행이 무시된다. 줄을 넘기기 위해 `\` 사용하지 않고도 여러 줄에 걸쳐 작성 가능

2.2.2 지능형 리스트와 map()/filter() 비교
지능형 리스트는 항목을 필터링 및 변환함으로써 시퀀스나 기타 반복 가능한 자료형으로부터 리스트를 만든다.

내장된 filter()와 map() 함수를 사용해서 이와 동일한 작업을 수행할 수는 있지만 가독성은 떨어진다. (5장)

map()과 filter() 함수를 이용해서 수행할 수 있는 작업은 기능적으로 문제가 있는 파이썬 lambda를 억지로 끼워 넣지 않고도 지능형 리스트를 이용해서 모두 구현할 수 있다.

In [3]:
# 에제 2-3) 지능형 리스트와 map()/filter() 구성으로 만든 동일 리스트
symbols = '$#(@&$'
beyond_ascii_1 = [ord(s) for s in symbol if ord(s) > 127]
beyond_ascii_2 = list(filter(lambda c: c > 127, map(ord, symbols)))

print(beyond_ascii_1)
print(beyond_ascii_2)

[]
[]


2.2.3 데카르트 곱

지능형 리스트는 두 개 이상의 반복 가능ㅎ나 자료형의 데카르트 곱을 나타내는 일련의 리스트를 만들 수 있다. ex) 그림 2-2

데카르트 곱 안에 들어 있는 각 항목은 입력으로 받은 반복 가능한 데이터의 각 요소에서 만들어진 튜블로 구성

생성된 리스트의 길이는 입력으로 받은 반복 가능 데이터의 길이와 동일

In [4]:
# 예제 2-4) 지능형 리스트를 이용한 데카르트 곱
colors = ['black', 'white']
sizes = ['S', 'M', 'L']

# 먼저 color를 반복하고 그 안에서 size를 반복해서 만든 튜플 리스트를 생성
# Tip) 지능형 리스트 안에서 줄을 바꾸면 생성될 리스트가 어떻게 정렬될지 알아보기 더 쉽다.
tshirts = [(color, size) for color in colors
                         for size in sizes]

print(tshirts)

for color in colors:
    for size in sizes:
        print((color, size))
        
# 먼저 size를 반복하고 그 안에서 color를 반복하려면 for 문의 순서만 바꾸면 된다.        
tshirts = [(color, size) for size in sizes
                         for color in colors]

print(tshirts)

[('black', 'S'), ('black', 'M'), ('black', 'L'), ('white', 'S'), ('white', 'M'), ('white', 'L')]
('black', 'S')
('black', 'M')
('black', 'L')
('white', 'S')
('white', 'M')
('white', 'L')
[('black', 'S'), ('white', 'S'), ('black', 'M'), ('white', 'M'), ('black', 'L'), ('white', 'L')]


2.2.4 제너레이터 표현식

지능형 리스트는 오로지 리스트를 만들때만 사용

다른 종류의 시퀀스를 채우려면 제너레이터 표현식을 사용

시퀀스형을 초기화하려면 지능형 리스트를 사용할 수도 있지만, 다른 생성자에 전달할 리스트를 통째로 만들지 않고 반복자 프로토콜(iterator protocol)을 이용해서 항목을 하나씩 생성하는 제너레이터 표현식은 메모리를 더 적게 사용

제너레이터 표현식은 지능형 리스트와 동일한 구문을 사용하지만, 대괄호 대신 괄호를 사용

In [5]:
# 예제 2-5) 제너레이터 표현식에서 튜플과 배열 초기화하기
symbols = '$#(@&$'

# 제너레이터 표현식이 함수에 보내는 단 하나의 인수라면 괄호 안에 또 괄호를 넣을 필요 없음
print(tuple(ord(symbol) for symbol in symbols))

import array
# 배열 생성자는 인수를 두 개 받으므로 제너레이터 표현식 앞뒤에 반드시 괄호를 넣어야 한다.
# 배열 생성자의 첫 번째 인수는 배열에 들어 갈 숫자들을 저장할 자료형을 지정 (2.9.1 참조)
print(array.array('I', (ord(symbol) for symbol in symbols)))

(36, 35, 40, 64, 38, 36)
array('I', [36, 35, 40, 64, 38, 36])


제너레이터 표현식은 한 번에 한 항목을 생성할 수 있도록 for 루프에 데이터를 전달하기 때문에 전체 항목을 메모리에 생성하지 않는다.

여기서는 제너레이터 표현식을 단지 리스트 이외의 시퀀스를 초기화하거나, 메모리에 유지할 필요가 없는 데이터를 생성하기 위해 사용하는 방법만을 보여주었다. 제너레이터가 작동하는 방식은 14장에서 자세히 설명한다.

In [6]:
# 예제 2-6) 제너레이터 표현식에서의 데카르트 곱
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


2.3 튜플은 단순한 불변 리스트가 아니다

튜플은 불변 리스트로 사용할 수도 있지만 필드명이 없는 레코드로 사용할 수도 있다.

2.3.1 레코드로서의 튜플

튜플은 레코드를 담고 있다. 튜플의 각 항목은 레코드의 필드 하나를 의미하며 항목의 위치가 의미를 결정한다.

튜플을 단지 불변 리스트로 생각한다면 경우에 따라 항목의 크기와 순서가 중효할 수도 있고 그렇지 않을 수도 있다.

튜플을 필드의 집합으로 사용하는 경우 항목 수가 고정되어 있고 항목의 순서가 중요하다. 튜플 안에서 항목의 위치가 항목의 의미를 나타내브로 튜플을 정렬하면 정보가 파괴된다는 점에 주의하라.

In [7]:
# 예제 2-7) 레코드로 사용된 튜플

lax_coordinates = (33.9425, -118.408056)    # 로스앤젤레스 국제공항의 위도와 경도

city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014) # 도쿄에 대한 데이터(지명, 년도, 백만 단위 인구수, 인구 변화율, 제곱킬로미터 단위 면적)

traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'CDA205856')] # (국가 코드, 여권 번호) 형태의 튜플로 구성된 리스트
for passport in sorted(traveler_ids):   # 리스트를 반복할 때 passport 변수가 각 튜플에 바인딩된다.
    print('%s/%s' % passport)           # % 포맷 연산자는 튜플을 이해하고 각 항목을 하나의 필드로 처리한다.
for country, _ in traveler_ids:         # 언패킹으로 튜플의 각 항목을 가져옴. 여기서 두 번째 항목에는 관심이 없으므로 더미 변수(_)에 할당.
    print(country)

BRA/CE342567
ESP/CDA205856
USA/31195855
USA
BRA
ESP


2.3.2 튜플 언패킹

튜플은 언패킹 메커니즘 덕분에 레코드로도 잘 작동한다. (tuple unpacking)

Tip) 튜플 언패킹(혹은 반복형 언패킹(iterable unpacking))은 반복 가능한 객체라면 어느 객체든 적용할 수 있다. 초과된 항목을 잡기 위해 *를 사용한 경우가 아니라면 반복 가능한 객체는 한 번에 하나의 항목을 생성한다.

튜플 언패킹은 병렬 할당(parallel assignment)을 할 때 가장 눈에 뛴다. 병렬 할당은 반복형 데이터를 변수로 구성된 튜플에 할당하는 것을 말한다.

In [8]:
# 병렬 할당
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates

# 튜블 언패킹을 이용하면 임시 변수를 사용하지 않고도 두 변수의 값을 서로 교환 할 수 있다.
a, b = 1, 2
b, a = a, b

# 함수를 호출할 때 인수 앞에 *를 붙여 튜플을 언패킹할 수 있다.
print(divmod(20, 8))
t = (20, 8)
print(divmod(*t))
quotient, remainder = divmod(*t)
print(quotient, remainder)

# 함수에서 호출자에 여러 값을 간단히 반환하는 기능
# 예를 들어 다음과 같이 더미 변수를 플레이스홀더로 사용해서 관심 없는 부분은 언패킹할 때 무시하여 파일시스템 경로에서 경로명과 파일 명을 가져올 수 있다.
import os
_, filename = os.path.split('/home/ajy8456/.ssh/python.py')
print(filename)

(2, 4)
(2, 4)
2 4
python.py


초과 항목을 잡기 위해 * 사용하기

튜플을 언패킹할 때 일부 항목에만 관심이 있는 경우에는 *를 사용할 수도 있다.

함수 매개변수에 *를 연결해서 초과된 인수를 가져우는 방법은 파이썬의 고전적인 기능이다.

파이썬 3에서는 이 개념을 확장해서 다음과 같이 병렬 할당에도 적용한다.

In [9]:
a, b, *rest = range(5)
print(a, b, rest)

a, b, *rest = range(3)
print(a, b, rest)

a, b, *rest = range(2)
print(a, b, rest)

0 1 [2, 3, 4]
0 1 [2]
0 1 []


In [10]:
# 병렬 할당의 경우 *는 단 하나의 변수에만 적용할 수 있다. 하지만 다음과 같이 어떠한 변수에도 적용할 수 있다.

a, *body, c, d = range(5)
print(a, body, c, d)

*head, b, c, d = range(5)
print(head, b, c, d)

0 [1, 2] 3 4
[0, 1] 2 3 4


2.3.3 내포된 튜플 언패킹

튜플 언패킹은 내포된 구조체에도 적용할 수 있다.

언패킹할 표현식을 받는 튜플은 (a, b, (c, d))처럼 다른 튜플을 내포할 수 있으며, 파이썬은 표현식이 내포된 구조체에 일치하면 제대로 처리한다.

In [11]:
# 예제 2-8) longitude에 접근하기 위해 내포된 튜플 언패킹하기

metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas:
    if longitude <= 0:
        print(fmt.format(name, latitude, longitude))

                |   lat.    |   long.  
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


주의) 파이썬 이전에는 `def fn(a, (b, c), d):` 처럼 매개변수 안에 내포된 튜플을 지정해서 함수를 정의할 수 있었지만, 파이썬 3부터 지원하지 않는다. 이는 함수를 호출하는 사용자 입장에서는 바뀐 점이 없고, 함수를 정의하는 방식에만 제한을 두는 것이다.

2.3.4 명명된 튜플

하지만 튜플을 레코드로 사용하기에는 부족한 점이 있다. 필드에 이름을 붙일 필요가 있다. 그래서 `namedtuple()` 함수가 고안되었다.

`collections.namedtuple()` 함수는 필드명과 클래스명을 추가한 튜플의 서브클래스를 생성하는 팩토리 함수로서, 디버깅할 때 유용하다.

필드명이 클래스에 저장되므로 `namedtuple()`로 생성한 객체는 튜플과 동일한 크기의 메모리만 사용한다. 속성을 객체마다 존재하는 `__dict__`에 저장하지 않으므로 일반적인 객체보다 메모리를 적게 사용한다.

In [12]:
#  예제 2-9) 명명된 튜플형을 정의하고 사용하기

from collections import namedtuple
from secrets import token_bytes

# 명명된 튜플을 정의하려면 클래스명과 필드명의 리스트 등 총 2개의 매개변수가 필요하다.
# 필드명의 리스트는 반복형 문자열이나 공백으로 구분된 하나의 문자열을 이용해서 지정한다.
City = namedtuple('City', 'name country population coordinates')

# 데이터는 위치를 맞추고 콤마로 구분해서 생성자에 전달해야 한다.
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))

# 필드명이나 위치를 이용해서 필드에 접근할 수 있다.
print(tokyo)
print(tokyo.population)
print(tokyo.coordinates)
print(tokyo[1])

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))
36.933
(35.689722, 139.691667)
JP


In [14]:
# 명명된 튜플은 튜플에서 상속받은 속성 외에 몇 가지 속성을 더 가지고 있다. ex) _fields 클래스 속성, _make(iterable) 클래스 매서드, _asdict() 객체 메서드

# 예제 2-10) 명명된 튜플의 속성과 메서드(예제 2-9)에 이어서)

print(City._fields) # _fields()는 클래스의 필드명을 담고 있는 튜플이다.
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
delhi = City._make(delhi_data)  # _make()는 반복형 객체로부터 명명된 튜플을 만든다. City(*delhi_data)를 호출하는 코드와 동일한 역할을 수행한다.
print(delhi._asdict())  # _asdict()는 명명된 튜플 객체에서 만들어진 collections.OrderedDict 객체를 반환한다.
# 이 메서드를 이용하면 레코드 데이터를 멋지게 출력할 수 있다.
for key, value in delhi._asdict().items():
    print(key + ':', value)

('name', 'country', 'population', 'coordinates')
OrderedDict([('name', 'Delhi NCR'), ('country', 'IN'), ('population', 21.935), ('coordinates', (28.613889, 77.208889))])
name: Delhi NCR
country: IN
population: 21.935
coordinates: (28.613889, 77.208889)


2.3.5 불변 리스트로서의 튜플

튜플을 불변 리스트로 사용할 때, 튜플과 리스트가 얼마나 비슷한지 알고 있으면 도움이 된다. (표 2-1)

항목을 추가하거나 삭제하는 기능 및 `__reversed__()` 메서드를 제외하고 리스트가 제공하는 메서드를 모두 지원한다.

2.4 슬라이싱

파이썬이 제공하는 list, tuple, str, 그리고 모든 시퀀스 형은 슬라이싱(slicing) 연산을 지원한다. 슬라이싱 연산은 대부분의 사람이 알고 있는 것보다 더 강력하다.

이 절에서는 고급 슬라이싱 형태의 사용법을 설명한다. 사용자 정의 클래스에서 슬라이싱을 구현하는 방법은 10장에서 설명한다.

2.4.1 슬라이스와 범위 지정시에 마지막 항목이 포함되지 않는 이유

슬리이스와 범위 지정시에 마지막 항목을 포함하지 않는 관례는 인덱스 번호가 0번부터 시작하는 파이썬, C 등의 언어에서 잘 작동한다. 이 관례 덕분에 다음과 같은 장점이 있다.

- 세 개의 항목을 생성하는 `range(3)`나 `my_list[:3]`처럼 중단점만 이용해서 슬라이스나 범위를 지정할 때 길이를 계산하기 쉽다.
- 시작점과 중단점을 모두 지정할 때도 길이를 계산하기 쉽다. 단지 중단점에서 시작점을 빼면 된다.
- x 인덱스를 기준으로 겹침 없이 시퀀스를 분할하기 쉽다. 단지 `my_list[:x]`와 `my_list[x:]`로 지정하면 된다.

2.4.2 슬라이스 객체

`s[a:b:c]`는 c 보폭(stride)만큼씩 항목을 건너뛰게 만든다. 보폭이 음수인 경우에는 거꾸로 거슬러 올라가 항목을 반환한다.

In [15]:
s = 'bicycle'
print(s[::3])
print(s[::-1])
print(s[::-2])

bye
elcycib
eccb


`a:b:c` 표기법은 인덱스 연산을 수행하는 `[]`안에서만 사용할 수 있으면 `slice(a, b, c)`객체를 생성한다.

`seq[start:stop:step]`표현식을 평가하기 위해 파이썬은 `seq.__getitem__(slice(start, stop, step))`을 호출한다. (10.4.1 참조)

시퀀스형을 직접 구현하지 않더라도 슬라이스 객체를 알아두면 도움이 된다. 스프레드시트에서 셀 범위에 이름을 붙이는 것처럼 슬라이스 객체는 슬라이스에 이름을 붙일 수 있게 해주기 때문이다.

In [16]:
# 예제 2-11) 단순 텍스트 파일 청구서의 행 항목들

from concurrent.futures.process import _system_limits_checked


invoice = """
0.....6.................................40........52...55........
1909  Pimoroni PiBrella                     $17.50    3    $52.50
1489  6mm Tactile Switch x20                 $4.95    2     $9.90
1510  Panavise Jr. - PV-201                 $28.00    1    $28.00
1601  PiTFT Mini Kit 320X240                $34.95    1    $34.95
"""

SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split('\n')[2:]
# 각 슬라이스에 이름을 붙여 for 루프의 가독성이 좋아짐
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

    $17.50   Pimoroni PiBrella                 
     $4.95   6mm Tactile Switch x20            
    $28.00   Panavise Jr. - PV-201             
    $34.95   PiTFT Mini Kit 320X240            
 


2.4.3 다차원 슬라이싱과 생략 기호

사용자 관점에서 보면 슬라이싱할 때는 다차원 슬라이스 및 생략 기호(...) 표기법 등의 기능을 사용할 수 있다.

[] 연산자는 콤마로 구분해서 여러 개의 인덱스나 슬라이스를 가질 수 있다. 이 방법은 예를들어 `NumPy` 외부 패키지에서 `a[i, j]` 구문으로 2차원 numpy.ndarray 배열의 항목이나 `a[m:n, k:l]` 구문으로 2차원 슬라이스를 가져올 때 사용한다.

[] 연산자를 처리하는 `__getitem__()`과 `__setitem__()` 특수 메서드는 `a[i, j]`에 들어 있는 인덱스를 튜플로 받는다. 즉, `a[i, j]`를 평가하기 위해 파이썬은 `a.__getitem__((i, j))`를 호출한다.

파이썬에 내장된 시퀀스형은 1차원이므로 단 하나의 인덱스나 슬라이스만 지원하고 튜플은 지원하지 않는다.

세 개의 마침표(...)로 표현된 생략 기호는 파이썬 파서에 의해 하나의 토큰으로 인식된다. 이 기호는 `Ellipsis`객체의 별명으로서 하나의 `ellipsis` 클래스의 객체다. 생략 기호 객체는 `f(a, ..., z)`처럼 함수의 인수나, `a[i: ...]`처럼 슬라이스의 한 부분으로 전달할 수 있다. NumPy는 다차원 배열을 슬라이싱할 때 생략 기호를 사용한다. 예를들어 x가 4차원 배열이라면 x[i, ...]는 x[i, :, :, :]와 동일하다.

현재(Python 3.5) 파이썬 표준 라이브러리에서 Ellipsis나 다차원 인덱스 및 슬라이스를 사용하는 사례는 보지 못했다. 이 구문법은 사용자 정의 자료형이나 NumPy 등의 확장 패키지는 지원하기 위해 존재하는 것으로 보인다.

2.4.4 슬라이스에 할당하기

슬라이스는 시퀀스에서 정보를 추출할 뿐만 아니라 가변 시퀀스의 값을 변경(즉, 시퀀스를 새로 만드는 것이 아니라 일부 항목의 값을 시퀀스 안에서 직접 변경함)할 때에도 사용할 수 있다.

할당문의 왼쪽에 슬라이스 표기법을 사용하거나 `del` 문의 대상 객체로 지정함으로써 가변 시퀀스를 연결하거나, 잘라 내거나, 값을 변경할 수 있다.

In [17]:
l = list(range(10))
print(l)

# 항목의 개수가 맞지 않아도 된다.
l[2:5] = [20, 30]
print(l)

del l[5:7]
print(l)

l[3::2] = [11, 22]
print(l)

l[2:5] = [100]  # 할당문의 대상이 슬라이스인 경우, 항목 하나만 할당하는 경우에도 할당문 오른쪽에는 반복 가능한 객체가 와야한다.
print(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 8, 9]
[0, 1, 20, 11, 5, 22, 9]
[0, 1, 100, 22, 9]


2.5 시퀀스에 덧셈과 곱셉 연산자 사용하기

어느 자료형의 시퀀스든 연결 연산이 많이 사용된다.

파이썬은 시퀀스가 덧셈(+)과 곱셈(*)을 지원한다.

일반적인 경우 덧셈의 경우 피연산자 두 개가 같은 자료형이어야 하며, 둘 다 변경되지 않지만 동일한 자료형의 시퀀스가 새로 만들어진다.

하나의 시퀀스를 여러 번 연결하려면 정수를 곱해서 표현한다. 이때에도 새로운 시퀀스가 만들어진다.

덧셈 및 곱셈 연산자는 언제나 객체를 새로 만들고, 피연산자를 변경하지 않는다.

2.5.1 리스트의 리스트 만들기

내포된 리스트를 가진 리스트를 초기화해야 하는 경우가 종종 있다. 이때 곱셉 연산자를 사용하려면 주의가 필요하다.

예제 2-12)처럼 지능형 리스트를 사용하는 것이 가장 좋다.

주의) a가 가변 항목을 담고 있을때 `a * n`과 같은 표현식을 사용하려면 주의해야 한다. 예를 들어 리스트의 리스트를 초기화할 때 `my_list = [[]] * 3`으로 초기화 하면 동일한 내부 리스트에 대한 참조 세 개를 가진 리스트가 만들어지므로 원치 않는 결과가 나올 수 있다.

참조와 가변 객체에 대한 메커니즘과 주의할 점은 8장 참조

In [21]:
# 예제 2-12) 길이가 3인 리스트 3개로 표현한 틱택토 보드
board = [['_'] * 3 for i in range(3)]
print(board)

board[1][2] = 'X'
print(board)

# 예제 2-13) 동일한 리스트에 대한 세 개의 참조를 가진 리스트는 쓸모 없다.
weird_board = [['_'] * 3] * 3   # 최상위 리스트가 동일한 내부 리스트에 대한 참조 세 개를 가져버린다.
print(weird_board)

weird_board[1][2] = 'O'
print(weird_board)

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]
[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', 'O'], ['_', '_', 'O'], ['_', '_', 'O']]


In [23]:
# 예제 2-13의 코드는 다음과 같이 작동한다.
row = ['_'] * 3
board = []
for i in range(3):
    board.append(row)   # 동일한 행이 board 리스트에 세 번 추가된다.
print(board)
board[2][0] = 'O'
print(board)

# 예제 2-12의 지능형 리스트는 다음 코드와 같이 작동한다.
board = []
for i in range(3):
    row = ['_'] * 3     # 반복할 때마다 row 객체를 새로 만들어서 board에 추가한다.
    board.append(row)
print(board)
board[2][0] = 'X'
print(board)

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['O', '_', '_'], ['O', '_', '_'], ['O', '_', '_']]
[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', '_'], ['_', '_', '_'], ['X', '_', '_']]


2.6 시퀀스의 복합 할당

복합 할당(augmented assignment)는 타깃 시퀀스의 가변성에 따라 상당히 다른 결과를 가져온다.

`+=`과 `*=` 등의 복합 할당 연산자는 첫 번째 피연산자에 따라 상당히 다르게 작동한다.

`+=` 연산자가 작동하도록 만드는 특수 메서드는 `__iadd__()`다. (메서드명에서 i는 in-place를 의미하며 해당 변수를 직접 변경한다.) 그런데 `__iadd__()` 메서드가 구현되어 있지 않으면, 파이썬은 대신 `__add__()` 메서드를 호출한다.

`a += b`에 대한 설명:

- a가 `__iadd__()` 메서드를 구현하면 구현되 메서드가 호출된다.
- a가 list, bytearray, arrary.array 등 가변 시퀀스인 경우 a의 값이 변경된다. (이 과정은 `a.extend(b)`와 비슷하다.)
- a가 `__iadd__()` 메서드를 구현하지 않는 경우 `a += b`표현식은 `a = a + b`가 되어 먼저 `a + b`를 평가하고, 객채를 새로 생성한 후 a에 할당된다.
- 즉, `__iadd__()` 메서드 구현 여부에 따라 a 변수가 가리키는 객체의 정체성이 바뀔 수도 있고 바뀌지 않을 수도 있다.

일반적으로 가변 시퀀스에 대해서는 `__iadd__()` 메서드를 구현해서 `+=` 연산자가 기존 객체의 내용을 변경하게 만드는 것이 좋다. 불변 시퀀스의 경우에는 이 연산을 수행할 수 없다.

`*=` 연산자의 경우에는 `__imul__()` 메서드를 통해 구현된다. (13장 참조)

Tip) 불변 시퀀스에 대해서는 새로운 항목을 추가하는 대신 항목이 추가된 시퀀스 전체를 새로 만들어 타깃 변수에 저장하므로, 불변 시퀀스에 반복적으로 연결 연산을 수행하는 것은 비효율적이다. 다만 str 객체의 작동 방식은 다르다. += 연산자를 이용해서 문자열을 만드는 작업을 빈번히 수행하므로 CPython은 이런 용법에 최적화되어 있다. str 객체는 메모리 안에 여분의 공간을 갖고 할당되므로 str 객체를 연결할 때 매번 전체 문자열을 다시 생성하지 않는다.

In [24]:
l = [1, 2, 3]
print(id(l))

l *= 2
print(l)
print(id(l))    # 기존 리스트 객체와 같은 객체

t = (1, 2, 3)
print(id(t))

t *= 2
print(t)
print(id(t))    # 새로운 튜플 객체가 생성

2137321324936
[1, 2, 3, 1, 2, 3]
2137321324936
2137320924568
(1, 2, 3, 1, 2, 3)
2137317512008


2.6.1 += 복합 할당 퀴즈

Q.

t = (1, 2, [30, 40])

t[2] += [50, 60]

A. t는 (1, 2, [30, 40, 50, 60])이 된다. & '튜플 객체는 항목 할당을 지원하지 않는다'는 메세지와 함께 TypeError가 발생한다.

In [26]:
# 예제 2-16) s[a] += b 표현식에 대한 바이트 코드

import dis
dis.dis('s[a] += b')

#  7: s[a] 값을 스택의 꼭대기(Top of Stack(TOS))에 놓는다.
# 11: TOS += b 연산을 수행한다. TOS가 가변 객체를 가리키면 이 연산은 성공한다.
# 13: TOS를 s[a]에 할당한다. s가 불변 객체면 이 연산은 실패한다.

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


이 코드에서는 다음 세 가지 교훈을 얻을 수 있다.

- 가변 항목을 튜플에 넣는 것은 좋은 생각이 아니다.
- 복합 할당은 원자적인(쪼개지지 않는) 연산이 아니다(앞의 예제에서 일부 연산이 수행된 후 예외가 발생한다.)
- 파이썬 바이트코드를 살펴보는 것은 그리 어렵지 않으며, 내부에서 어떤 일이 발생하고 있는지 살펴보는데 도움이 된다.

2.7 list.sort()와 sorted() 내장 함수

list.sort():
- 메서드는 사본을 만들지 않고 리스트 내부를 변경해서 정렬한다.
- 새로운 리스트를 생성하지 않았음을 알려주기 위해 `None`을 반환한다.

파이썬 API의 관례: 객체를 직접 변경하는 함수나 메서드는 객체가 변경되었고 새로운 객체가 생성되지 않았음을 알려주기 위해 None을 반환 ex) list.sort(), random.shuffle()

NOTE) 위의 관례는 메서드를 연결해서 호출할 수 없는 단점이 있다. 이와는 반대로 str 객체의 메서드들처럼 새로운 객체를 반환하는 메서드는 플루언트 인터페이스(Fluent Interface) 스타일(ex. method chaining)로 메서드를 연결할 수 있다.

sorted():
- 새로운 리스트를 생성해서 반환한다.
- 불변 시퀀스 및 제너레이터를 포함해서 반복 가능한 모든 객체를 인수로 받을 수 있다. (14장 참조)
- 입력받은 반복 가능한 객체의 자료형과 무관하게 언제나 새로 생성한 `list`를 반환한다.

둘 다 선택적으로 두개의 keyword를 인수로 받는다.
- reverse: default = False
- key: 정렬에 사용할 키를 생성하기 위해 각 항목에 적용할 함수를 인수로 받는다. 키를 지정하지 않으면 항목 자체를 비교한다. (Tip. 내장 함수 및 표준 라이브러리 함수도 사용할 수 있다.)

파이썬은 정렬 알고리즘으로 팀정렬(Timsort)를 사용한다. 이는 안정적(비교해서 동일한 항목들이 상대적인 순서를 유지)이다.

Tip) 시퀀스를 정렬한 후에는 아주 효율적으로 검색 및 삽입 등을 할 수 있다. 또한, 정렬은 값비싼 연산이므로 시퀀스를 일단 정렬한 후에는 정렬 상태를 유지하는 것이 좋다.

In [2]:
# 키워드 인수를 사용하는 방법을 알아보기 위한 예시

fruits = ['grape', 'raspberry', 'apple', 'banana']
list_1 = sorted(fruits)     # 알파벳순으로 정렬된 문자열들을 담은 새로운 리스트를 생성
print(list_1)
print(fruits)               # 타깃 객체를 변환하지는 않는다.

list_2 = sorted(fruits, reverse=True)
print(list_2)

list_3 = sorted(fruits, key=len)    # 문자열 길이에 따라 정렬된 새로운 리스트 반환
print(list_3)                       # 정렬 알고리즘이 안정적이므로 길이가 똑같이 5인 'grape'와 'apple'은 원래 순서가 유지

list_4 = sorted(fruits, key=len, reverse=True)
print(list_4)                       # 정렬 알고리즘이 안정적이므로 list_3에 정확히 역순이 아니다.

list_5 = fruits.sort()
print(list_4)                       # None을 반환
print(fruits)                       # 타깃 리스트가 정렬됨

['apple', 'banana', 'grape', 'raspberry']
['grape', 'raspberry', 'apple', 'banana']
['raspberry', 'grape', 'banana', 'apple']
['grape', 'apple', 'banana', 'raspberry']
['raspberry', 'banana', 'grape', 'apple']
['raspberry', 'banana', 'grape', 'apple']
['apple', 'banana', 'grape', 'raspberry']


2.8 정렬된 시퀀스를 bisect로 관리하기

bisect 모듈은 bisect()와 insort() 함수를 제공한다.

bisect()는 이진 검색 알고리즘을 이용해 시퀀스를 검색하고, insort()는 정렬된 시퀀스 안에 항목을 삽입한다.

2.8.1 bisect()로 검색하기

bisect(haystack, needle)은 정렬된 시퀀스인 haystack 안에서 오름차순 정렬 상태를 유지한 채로 needle을 추가할 수 있는 위치를 찾아낸다. 즉, 해당 위치 앞에는 needle보다 같거나 작은 항목이 온다.

bisect()의 결과값을 index로 사용해서 haystack.insert(index, needle)을 호출하면 needle을 추가할 수 있지만, insort() 함수는 이 두 과정을 더 빨리 처리한다.

Tip) SortedCollection(http://bit.ly/1Vm6WEa)에서는 bisect 모듈을 사용하지만 이런 함수들을 따로 사용하는 것보다 훨씬 쉽게 정렬하는 비법을 제공하고 있다.

bisect의 행동은 두가지 방식으로 조절할 수 있다.
- 선택 인수인 lo와 hi를 사용하면 삽입할 때 검색할 시퀀스의 영역을 좁힐 수 있다. lo의 기본값은 0, hi의 기본값은 시퀀스의 len()이다.
- bisect는 실제로는 bisect_right() 함수의 별명이며, 이 함수의 자매 함수로 bisect_left()가 있다. 이 두 함수는 단지 리스트 안의 항목이 needle과 값이 같을 때만 차이가 난다. bisect_right()는 기존 항목 바로 뒤를 삽입 위치로 반환하며, bisect_left()는 기존 항목 위치를 삽입 위치로 반환하므로 기존 항목 바로 앞에 삽입된다.

In [5]:
# 예제 2-17 정렬된 시퀀스에서 항목을 추가할 위치를 찾아내는 bisect

import bisect
import sys
from turtle import position

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)              # 삽입 위치를 찾아내기 위해 선택한 bisect 함수를 사용한다.
        offset = position * '  |'                           # 간격(offset)에 비례해서 수직 막대 패턴을 만든다.
        print(ROW_FMT.format(needle, position, offset))     # needle과 삽입 위치를 보여주는 포맷된 행을 출력한다.

if __name__ == '__main__':
    if sys.argv[-1] == 'left':                              # prompt의 마지막 명령행 인수에 따라 사용할 bisect 함수를 선택한다.
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect

print('DEMO:', bisect_fn.__name__)                          # 선택된 함수명을 헤더에 출력한다.
print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
demo(bisect_fn)

DEMO: bisect_right
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 


In [7]:
# 예제 2-18) 시험 점수를 입력받아 등급 문자를 반환하는 grade() 함수
# 이 예제처럼 bisect를 사용하면 수치형 값으로 테이블을 참조할 수 있다.
# 정렬된 긴 숫자 시퀀스를 검색할 때 index() 대신 더 빠른 bisect() 함수를 사용하는 다른 여러 예제는 파이썬 공식 문서의 bisect 모듈 부분에서 찾아볼 수 있다.

def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]

[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

2.8.2 bisect.insort()로 삽입하기

정렬은 값비싼 연산이므로 정렬된 상태를 유지하는 것이 좋기 때문에 bisect.insort() 함수가 만들어졌다.

insort(seq, item)은 seq를 오름차순으로 유지한 채로 item을 seq에 삽입한다.

이 함수도 역시 선택적으로 lo와 hi 인수를 받아 시퀀스 안에서 검색할 범위를 제한할 수 있다. 또한, 삽입 위치를 검색하기 위해 bisect_left() 함수를 사용하는 insort_left() 함수도 있다.

In [8]:
# 예제 2-19 시퀀스를 항상 정렬된 상태로 유지하는 insort()

import bisect
import random

SIZE = 7

random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE * 2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


2.9 리스트가 답이 아닐 때

리스트형은 융통성 있고 사용하기 편해서 파이썬 프로그래머들은 리스트형을 남용하곤 한다.

하지만 세부 요구사항에 따라 더 나은 자료형도 있다. 예를 들어 만약 숫자들로 구성된 리스트를 다루고 있다면 배열을 사용하는 것이 좋다.

배열은 모든 기능을 갖춘 float 객체 대신 C언어의 배열과 마찬가지로 기계어 형태로 표현된 바이트 값만 저장하기 때문에 훨씬 더 효율적이다.

한편 리스트의 양쪽 끝에 항목을 계속 추가하거나 삭제하면서 FIFO나 LIFO 데이터 구조를 구현할 때에는 덱(deque, 양쪽을 사용하는 큐)가 더 빠르다.

Tip) item in my_collection처럼 어떤 항목이 들어있는지 검사하는 작업을 많이 수행하며, 특히 항목 수가 아주 많은 경우에는 my_collection을 set 형으로 구현하는 것을 고려해보자. set형은 항목이 들어 있는지 검사하는 과정이 최적화되어 있다. 그러나 집합은 순서가 없으며 시퀀스가 아니다. (3장 참조)

2.9.1 배열

리스트 안에 숫자만 들어 있다면 배열(array.array)이 리스트보다 훨씬 더 효율적이다.

배열은 가변 시퀀스가 제공하는 모든 연산을 지원하며, 빠르게 파일에 저장하고 읽어올 수 있는 frombytes()와 tofile() 메서드도 추가로 제공한다.

파이썬 배열은 C 배열만큼 가볍다.

배열을 생성할 때는 배열에 저장되는 각 항목의 C 기반 형을 결정하는 문자인 타입코드(typecode)를 지정한다. ex) signed char에 대한 타입코드는 b이다. 따라서 array('b') 배열을 생성하면 각 항목은 하나의 바이트로 저장되고 -128에서 127까지의 정수로 해석된다.

숫자가 아주 많이 들어 있는 시퀀스의 경우 배열에 저장하면 메모리가 많이 절약된다.

또한, 배열형에 맞지 않는 숫자를 저장할 수 없게 한다.

In [11]:
# 예제 2-20 커다란 실수 배열의 생성, 저장, 로딩 - 훨씬 빠르고 적은 용량을 소모

from array import array
from random import random

floats = array('d', (random() for _ in range(10 ** 7)))
print(floats[-1])

fp = open('floats.bin', 'wb')
floats.tofile(fp)
fp.close()

floats2 = array('d')
fp = open('floats.bin', 'rb')
floats2.fromfile(fp, 10 ** 7)
fp.close()

print(floats2[-1])
print(floats2 == floats)

0.1288579230853678
0.1288579230853678
True


Tip) 객체를 직렬화하는 pickle 모듈도 숫자 데이터를 빠르고 융통성 있게 저장할 수 있다. pickle.dump() 메서드는 실수 배열을 array.tofile() 메서드만큼 빠르게 저장할 뿐만 아니라 복소수, 내포된 컬렉션, (저장하기 까다롭게 구현된 경우가 아니라면) 심지어 사용자 정의 객체 등 거의 모든 내장 자료형을 처리할 수 있다.

래스터 이미지(raster images)처럼 이진 데이터를 표현하는 숫자 배열을 위해 파이썬은 bytes와 bytearray 형을 제공한다. (4장 참조)

표 2-2: 리스트와 배열의 기능을 비교

Tip) 파이썬 3.5까지 array형은 list.sort()처럼 배열을 직접 변경하는 메서드가 없다. 배열을 정렬하려면 sorted() 함수를 호출하고 배열을 다시 만들어야 한다. 정렬된 배열을 유지하면서 항목을 추가할 때에는 bisect.insort() 함수를 사용할 수 있다.

2.9.2 메모리 뷰

메모리 뷰(memoryview) 내장 클래스는 공유 메모리 시퀀스형으로서 bytes를 복사하지 않고 배열의 슬라이스를 다룰 수 있게 해준다. 이 클래스는 NumPy 라이브러리에서 영감을 받아 만들었다.

메모리 뷰는 본질적으로 (math를 포함하지 않은) 파이썬 자체에 들어 있는 NumPy 배열 구조체를 일반화 한 것이다. 메모리 뷰는 PIL 이미지, SQLite 데이터베이스, NumPy 배열 등 데이터 구조체를 복사하지 않고 메모리를 공유할 수 있게 해준다. 데이터셋이 커지는 경우 이것은 아주 중요한 기법이다.

array 모듈과 비슷한 표기법을 사용하는 memoryview.cast() 메서드는 바이트를 이동시키지 않고 C 언어의 형변환 연산자처럼 여러 바이트로 된 데이터를 읽거나 쓰는 방식을 바꿀 수 있게 해준다. memoryview.cast()는 또 다른 memoryview 객체를 반환아며 언제나 동일한 메모리를 공유한다.

In [14]:
# 예제 2-21 배열 항목 값의 바이트 중 하나를 변경하기

import array

numbers = array.array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)                          # 짧은 정수 타입 다섯 개가 있는 배열에서 memoryview 객체를 만든다.
print(len(memv))
print(memv[0])                                      # memv도 배열 안에 있는 5개의 항목을 동일하게 바라본다.

memv_oct = memv.cast('B')                           # memv 요소를 unsigned char로 형변환한 memv_oct를 생성한다.
print(memv_oct.tolist())                            # 값을 조사하기 위해 memv_oct 안의 요소를 리스트로 만든다.

memv_oct[5] = 4
print(numbers)                                      # 2바이트 unsigned int의 최상위 바이트에서 4는 1024에 해당한다.

5
-2
[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]
array('h', [-2, -1, 1024, 1, 2])


2.9.3 NumPy와 SciPy

NumPy와 SciPy가 제공하는 고급 배열 및 행렬 연산 덕분에 파이썬이 과학 계산 어플리케이션에서 널리 사용되게 되었다.

NumPy는 숫자뿐만 아니라 사용자 정의 레코드도 저장할 수 있는 다차원 동형 배열 및 행렬을 구현하고 요소 단위에서 효율적으로 연산할 수 있게 해준다. numpy.ndarray의 모든 요소를 저장, 로딩, 처리하는 고급 연산도 지원한다.

SciPy는 NumPy를 기반으로 작성된 라이브러리로서, 선형대수학, 수치해석, 통계학에 나오는 여러 과학 계산 알고리즘을 제공한다. Netlib repository가 제공하는 C 및 포트란 코드 기반을 활용함으로써 빠르고 신회성이 높다. 따라서 SciPy는 C와 포트란에서 최적화되고 업계에서 입증된 수치 계산 함수를 대화형 고급 파이썬 API를 통해 과학자들에게 제공한다.

NumPy와 SciPy는 막강한 라이브러리로서 Pandas와 Blaze 데이터 분석 라이브러리의 기반이기도 하다. 이 라이브러리들은 비수치형 데이터를 담은 효율적인 배열형 뿐만 아니라 .csv, .xls, SQL dump, HDF5 등 다양한 포맷과 호환되는 import/export 함수도 제공한다.

2.9.4 덱 및 기타 큐

append()와 pop() 메서트를 사용해서 리스트를 스택이나 큐(FIFO 방식으로 작동하는)로 사용할 수는 있다. 그러나 리스트 왼쪽(0번 인덱스)에 삽입하거나 삭제하는 연산은 전체 리스트를 이동시켜야 하므로 처리 부담이 크다.

덱(collections.deque) 클래스는 큐의 양쪽 어디에서든 빠르게 삽입 및 삭제할 수 있도록 설계된 thread-safe 양방향 큐이다.

덱은 최대 길이를 설정해서 제한된 항목만 유지할 수도 있으므로 덱이 꽉 찬 후에는 새로운 항목을 추가할 때 반대쪽 항목을 버린다. 예를들어 '최근에 본 항목'이나 이와 비슷한 것들의 목록을 유지할 때도 사용할 수 있다.

표 2-3) 리스트와 덱 고유의 메서드들을 비교: 덱은 리스트 메서드 대부분을 구현할 뿐만 아니라 popleft()와 rotate()처럼 고유한 메서드들을 추가로 가지고 있다.

하지만 덱의 중간 항목을 삭제하는 연산은 그리 빠르지 않다는 단점이 있다. 덱이 양쪽 끝에 추가나 제거하는 연산에 최적화 되어 있기 때문이다.

append()와 popleft() 메서드는 원자석을 갖고 있으므로 멀티스레드 앱에서 락을 사용하지 않고도 덱을 이용해서 간단히 FIFO 큐를 구현할 수 있다.

파이썬 표준 라이브러리 패키지에는 덱 이외에도 다음과 같은 큐를 구현하고 있다.
- queue: queue 모듈에서는 동기화된(즉, thread-safe) Queue, LifoQueue, PriorityQueue 클래스를 제공한다. 이 클래스들은 스레드 간에 안전하게 통신하기 위해 사용된다. 세 클래스 모두 0보다 큰 maxsize 인수를 생성자에 전달해서 바인딩 할 수 있다. 그렇지만 덱과 달리 공간이 꽉 찼을 때 항목을 버리지 않는다. 대신 새로운 항목의 추가를 블로킹하고 다른 스레드에서 큐 안의 항목을 제거해서 공간을 확보해줄 때까지 기다린다. 따라서 활성화된 스레드 수를 조절하기 좋다.
- multiprocessing: multiprocessing 모듈은 queue.Queue와 비슷하지만 프로세스 간 통신을 지원하기 위해 설계된 고유한 Queue 클래스를 구현한다. 태스크 관리에 특화된 multiprocessing.JoinableQueue 클래스도 제공한다.
- asyncio: 파이썬 3.4에 새로 추가된 asyncio 모듈은 queue 및 multiprocessing 모듈에 포함된 클래스로 부터 영감을 얻은 Queue, LifoQueue, PriorityQueue, JoinableQueue 클래스를 제공하지만, 비동기 프로그래밍 환경에서 작업을 관리하는 데 주안점을 두고 있다.
- heapq: 앞의 세 모듈과 대조적으로 heapq는 queue 클래스를 구현하지는 않지만, 가변 시퀀스를 heap queue나 priority queue로 사용할 수 있게 해주는 heappush()와 heappop()등의 함수를 제공한다.

In [1]:
# 예제 2-23) 덱 이용하기

from collections import deque

dq = deque(range(10), maxlen=10)    # 선택적 인수 maxlen은 덱 객체가 수용할 수 있는 최대 항목 수를 설정한다. 덱 객체를 생성할 때 읽기 전용 속선인 maxlen을 설정한다.
print(dq)

dq.rotate(3)                        # rotate() 메서드는 양수 인수를 받으면 오른쪽 끝에 있는 항목을 지정한 개수만큼 왼쪽 끝으로, 음수 인수를 받으면 왼쪽 끝에 있는 항목을 지정한 개수만큼 오른쪽 끝으로 이동시킨다.
print(dq)

dq.rotate(-4)
print(dq)

dq.appendleft(-1)                   # 가득한 찬 덱(len(dq) == dq.maxlen)에 항목을 추가하면 반대쪽 항목을 삭제한다.
print(dq)

dq.extend([11, 22, 33])
print(dq)

dq.extendleft([10, 20, 30, 40])
print(dq)

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)
deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)
deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33], maxlen=10)
deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8], maxlen=10)


2.10 요약

파이썬 시퀀스는 가변형과 불변형으로 구분하기도 하지만 균일 시퀀스와 컨테이너 시퀀스로 분류하기도 한다.

균일 시퀀스는 작고, 빠르고, 사용하기 쉽지만 숫자, 문자, 바이트처럼 원자적인 데이터만 저장할 수 있다.

컨테이너 시퀀스는 융통성이 있지만, 가변 객체를 저장할 때는 예상치 못한 일이 발생할 수도 있다. 따라서 내포된 데이터 구조체와 함께 컨데이너 시퀀스를 사용할 때는 주의해야한다.

지능형 리스트와 제너레이터 표현식은 시퀀스를 생성하고 초기화하는 강력한 표기법이다.

튜플은 익명 필트를 가진 레코드 및 불변 리스트로 사용할 수 있다. 튜플을 레코드로 사용할 때는 튜플 언패킹이 필드에 접근하는 가장 안전하고 가독성이 좋은 방법이다. *구문을 이용하면 일부 필드를 무시하거나 선택적 필드를 처리하기 상당히 좋다.

명명된 튜플은 튜플과 마찬가지로 객체당 오버헤드가 적으며, 이름을 이용해서 간단히 필드에 접근할 수 있고, _asdict() 메서드를 사용하면 레코드를 OrderedDict 객체로 export할 수도 있다.

시퀀스 슬라이싱은 파이썬에서 즐겨 사용하는 구문 기능으로서, 사람들이 흔히 알고 있는 것보다 훨씬 더 강력하다. NumPy에서 사용되는 다차원 슬라이싱과 생략기호(...) 표기도 사용자 정의 시퀀스에서 지원할 수 있다. 슬라이스에 할당하는 구문은 가변 시퀀스의 편집을 멋지게 표현한다.

seq * n으로 표현되는 반복 연결은 편리하게 사용할 수 있으며, 주의해서 사용하면 가변 항목을 담은 리스트의 리스트를 초기화할 수도 있다.

복합 할당 연산자는 가변/불변 시퀀스 여부에 따라 다르게 작동한다. 대상 시퀀스가 불변인 경우에는 새로운 시퀀스를 생성한다. 그러나 가변인 경우에는 대상 시퀀스를 직접 변경한다. 하지만 시퀀스 구현 방식에 따라 그렇지 않을 수도 있다.

sort() 메서드와 sorted() 내장 함수는 선택적인 key 인수에 정렬 기준을 계산하는 함수를 지정할 수 있다.

정렬된 시퀀스의 순서를 유지하면서 항목을 추가하려면 bisect.insort() 메서드를 사용하고, 정렬된 시퀀스를 효율적으로 검색하려면 bisect.bisect() 메서드를 사용하라.

파이썬 표준 라이브러리는 리스트와 튜플 외에 array.array도 제공한다. NumPy와 SciPy는 표준 라이브러리에 속해 있지 않지만, 대형 데이터셋에 수치 연산을 수행하는 경우에 큰 도움이 된다.

기능이 풍부하고 thread-safe한 collections.deque도 있다. 또한 표준 라이브러리에서 구현하는 여러 큐 클래스들에 대해 간략히 알아보았다.

2.11 읽을거리

다른 객체에 대한 참조를 담고 있는 객체도 있는데, 이런 객체를 컨테이너라고 한다.

혼함된 자료형 리스트: 파이썬 입문서들은 리스트를 여러 자료형의 객체를 담을 수 있다고 강조하지만, 사실 이 기능은 그리 쓸모 있지는 않다. 리스트에 항목을 넣는 이유는 대부분 나중에 항목을 처리하기 위한 것이므로, 모든 항목이 어떤 연산을 공통적으로 지원해야한다. 리스트와 달리 튜플은 다른 자료형을 담는 경우가 종종 있다. 이는 튜플에 들어 있는 각 항목이 실제로는 하나의 필드이며, 각 필드의 형은 서로 독립적이므로 자연스러운 현상이다.

sorted()와 list.sort()에 사용된 정렬 알고리즘은 팀정렬(Timsort)이다. 팀정렬은 데이터의 정렬된 정도에 따라 삽입 정렬과 병합 정렬 사이를 전환하는 적응형 알고리즘이다. 실세계 데이터에는 정렬된 데이터 덩어리들이 들어 있는 경우가 많기 때문에 상당히 효율적이다. 팀정렬은 CPython에서 처음으로 구현되었다.